In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from collections import defaultdict

from model.Base import Base
from model.Device import Device
from model.DeviceAppTraffic import DeviceAppTraffic
from model.DnsReq import DnsReq
from model.Flow import Flow
from model.User import User
from model.user_devices import user_devices

from sqlalchemy import create_engine, text, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.pool import NullPool

from IPython.display import display

#output_notebook()

DB='postgresql+psycopg2:///ucnstudy'

engine = create_engine(DB, echo=False, poolclass=NullPool)
Base.metadata.bind = engine
Session = sessionmaker(bind=engine) 

Analyze device usage per user

In [1]:
ses = Session()

users = ses.query(User)

for user in users:
    sql_user_devices = text('select * from user, user_devices where user_devices.user_id =:user').bindparams(user = user.id)
    user_devices = ses.execute(sql_user_devices)

    print ("user: " + str(user.id))
    for devices in user_devices:
        sql_domains_accessed = text('select dnsreqs.query_domain, count(*) from dnsreqs, user_devices where dnsreqs.devid =:devid group by dnsreqs.query_domain').bindparams(devid = devices.device_id)
        domains_accessed = ses.execute(sql_domains_accessed)
    print ("device: " + str(devices.device_id))
        dom = []
        accesses = []
        length = 0
        for domains in domains_accessed:
            #print (domains.query_domain, domains.count)
            dom.append(domains.query_domain)
            accesses.append(domains.count)
            length = length + 1
        x = np.arange(length)
        plt.bar(x, accesses)
        #plt.xticks(x + 0.5, dom, rotation = 90)
        plt.show()
        


IndentationError: unexpected indent (<ipython-input-1-4f870915f7f6>, line 14)

Analizing beginning and end of daily activities 

In [4]:
ses = Session()

devices = ses.query(Device).distinct().order_by(Device.id)

for dev in devices: 
    
    sql_beg_day = text("SELECT distinct devid, flows.startts FROM flows join (SELECT DATE(startts) as date_entere\
d, MIN(startts) as min_time FROM flows WHERE devid =:d_id and extract (hour from startts) > 3 GROUP BY date(startts)) AS grp ON grp.min_ti\
me = flows.startts order by flows.startts;").bindparams(d_id = dev.id)
    result_beg_day = ses.execute(sql_beg_day)
    
    info = defaultdict(list)
    for row in result_beg_day:
        info['devid'].append(str(row[0]))
        info['start'].append(row[1])
        
    sql_end_day = text("SELECT DISTINCT devid, flows.endts FROM flows join (SELECT DATE(endts) as date_entered, M\
AX(endts) as max_time FROM flows WHERE devid =:d_id GROUP BY date(endts)) AS grp ON grp.max_time = flo\
ws.endts order by flows.endts;").bindparams(d_id = dev.id)
    result_end_day = ses.execute(sql_end_day)

    for row in result_end_day:
        info['end'].append(row[1])
        
    #adding time usage before 3 am
    sql_startts = text("select devid, startts from flows where devid = :d_id order by startts").bindparams(d_id = dev.id)
    result_startts = ses.execute(sql_startts)
    
    for row in result_startts:
        timst = row[1]
        in_list = False
        for date in info['start']:
            if date.day == timst.day and date.month == timst.month and date.year == timst.year:
                in_list = True
        if in_list == False:
            info['start'].append(timst)
    
    info['start'].sort()
    table = pd.DataFrame(info)
    display(table)
    
    
    #creates table with times for each week day
    info_week = defaultdict(list)
    if (info['start']):
        for timst in info['start']:
            day = timst
            weekday = day.strftime('%A')
            key = weekday + ' start'
            info_week[key].append(day)
            #print(day, day.strftime('%A'))

        for timst in info['end']:
            day = timst
            weekday = day.strftime('%A')
            key = weekday + ' end'
            info_week[key].append(day)

        days_str = {'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'}
        for name in days_str:
            df_col = defaultdict(list)
            df_col['device'] = str(dev.id)
            df_col[name+' beg'] = info_week[name+' start']
            df_col[name+' end'] = info_week[name+' end']
            df_week = pd.DataFrame(df_col)
            #df_week.columns = [name+' start', name + ' end']
            display(df_week)
    
ses.close()

ValueError: arrays must all be same length